In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!pip install d2l

In [ ]:
%matplotlib inline
import torch
import torchvision
from torch import nn
from d2l import (
    torch as d2l,
)  # 把d2l中的torch包重新命名为d2l进行调用，有些方法被d2l的作者保存在了d2l的包中，并且用一个注释#@save指明了

d2l.set_figsize([4, 4])  # d2l定义的一个设置图片大小的函数,可以输入一个列表手动控制大小如[5,6]
content_img = d2l.Image.open("../input/imageshow/content.jpg")  # 从当前目录导入内容图片
print(content_img)  # 图片格式
d2l.plt.imshow(content_img)
# 按照指定格式展示图片，默认必须输入一个参数x，即图片内容参数

In [ ]:
style_img = d2l.Image.open("../input/imageshow/style3.jpg")  # 读入风格图片
print(style_img)  # 可以看出内容图片和风格图片大小是不一样的
d2l.plt.imshow(style_img)
# 展示风格图片


In [ ]:
rgb_mean = torch.tensor([0.485, 0.456, 0.406])  # 定义RGB图像的三个通道的均值
rgb_std = torch.tensor([0.229, 0.224, 0.225])  # 定义RGB图像的三个通道的方差

# 定义预处理函数，输入两个参数：图像和需要重新定义的图像形状大小
def preprocess(img, image_shape):
    # 使用torchvision的转换容器
    #   1、重新裁剪图像大小（神经网络只能处理同样大小的图像）
    #   2、转换成张量形式
    #   3、按照之前定义的进行归一化，避免某些像素过于大或者过于小影响训练效果（梯度下降速度等）
    transforms = torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize(image_shape),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=rgb_mean, std=rgb_std),
        ]
    )
    # unsqueeze主要起到升维的作用，后续图像处理可以更好地进行批操作，在最低维增加一个0维轴
    return transforms(img).unsqueeze(0)

#定义后处理函数，输入一个参数：图像
def postprocess(img):
    #将0维轴（即一个batch）的数据传入到对应的设备处理
    img = img[0].to(rgb_std.device)
    #将图像的CHW形式转换成卷积网络可以处理的HWC形式，并约束在方差和均值内，min=0，max=1，裁剪掉负数和大于1的值
    img = torch.clamp(img.permute(1, 2, 0) * rgb_std + rgb_mean, 0, 1)
    #将变回CHW形式的张量数据传递给PIL Image
    return torchvision.transforms.ToPILImage()(img.permute(2, 0, 1))

In [ ]:
#从torchvision的models库中导入VGG16模型，而且是已经经过参数训练过的
#VGG19抽取[0,5,10,17,21]层作为风格层，24层作为内容层
pretrained_net = torchvision.models.vgg16(pretrained=True)
style_layers, content_layers = [0,5,10,17,21], [24]

In [ ]:
#从torchvision的models库中导入VGG16模型，而且是已经经过参数训练过的
#VGG16抽取[0,2,5,7,10]，28层作为内容层
pretrained_net = torchvision.models.vgg16(pretrained=True)
style_layers, content_layers = [0,2,5,7,10], [28]

In [ ]:
#从torchvision的models库中导入VGG19模型，而且是已经经过参数训练过的
#VGG19抽取0、5、10、19、28层作为风格层，25层作为内容层
pretrained_net = torchvision.models.vgg19(pretrained=True)
style_layers, content_layers = [0,5,10,19,28], [25]

In [ ]:
print(pretrained_net)

In [ ]:
#新建一个用于我们自己风格化处理的网络模型，只需要用到我们需要的几个层
#只需要用到从输入层到最靠近输出层的内容层或样式层之间的所有层
#可以简单调用torchvision的features模块组合成一个新的网络
#作为实参的话，*相当于对tuple的解构，同样的**则是对dict的解构
net = nn.Sequential(
    *[pretrained_net.features[i] for i in range(max(content_layers + style_layers) + 1)]
)
# print(net)
# len(net)
# net[0]


In [ ]:
#提取特征函数，输入参数：图像的张量形式的量（即需要preprocess过），内容层序号，风格层序号
def extract_features(X, content_layers, style_layers):
    #初始化内容和风格
    contents = []
    styles = []
    for i in range(len(net)):
        #调用net[i]层，输入图像X，返回结果重新给到X，即进行层net[i]的卷积or池化or全连接处理
        X = net[i](X)
        #下面判断是属于风格层还是内容层
        if i in style_layers:
            #风格styles列表的内容扩展添加上X
            styles.append(X)
        if i in content_layers:
            #内容contents列表的内容扩展添加上X
            contents.append(X)
    return contents, styles


In [ ]:
# 获取内容图像的内容特征，输入参数：图像形状大小，调用的设备device="cpu" or "gpu"
def get_contents(image_shape, device):
    # 图像的预处理，完成了裁剪、张量转换、归一化三个操作
    content_X = preprocess(content_img, image_shape).to(device)
    # 同样可以调用之前的提取特征函数进行特征的提取，这是最原始的内容图像的内容特征提取
    # 这里只需要内容特征，不需要风格特征
    contents_Y, _ = extract_features(content_X, content_layers, style_layers)
    #X都是表示原始图像，Y表示进行了提取特征操作的图像
    return content_X, contents_Y


# 获取风格图像的风格特征，输入参数：图像形状大小，调用的设备device="cpu" or "gpu"
def get_styles(image_shape, device):
    # 图像的预处理，完成了裁剪、张量转换、归一化三个操作
    style_X = preprocess(style_img, image_shape).to(device)
    # 同样可以调用之前的提取特征函数进行特征的提取，这是最原始的风格图像的风格特征提取
    # 这里只需要风格特征，不需要内容特征
    _, styles_Y = extract_features(style_X, content_layers, style_layers)
    return style_X, styles_Y


In [ ]:
def content_loss(Y_hat, Y):
    # 我们从动态计算梯度的树中分离目标：
    # 这是一个规定的值，而不是一个变量。
    return torch.square(Y_hat - Y.detach()).mean()


In [ ]:
# 定义gram函数，输入为一个矩阵，默认为每个行向量之间的相关性
def gram(X):
    # 计算通道个数C=X.shape[1]，索引为1的轴的维度
    # 获取X的元素个数,X.numel()，再用个数除以通道个数得到HW的值，即列的个数
    num_channels, n = X.shape[1], X.numel() // X.shape[1]
    # 按照通道个数C和列的个数HW进行reshape
    X = X.reshape((num_channels, n))
    # 进行矩阵相乘算法，即Gram=XX^T
    return torch.matmul(X, X.T) / (num_channels * n)


In [ ]:
def style_loss(Y_hat, gram_Y):
    return torch.square(gram(Y_hat) - gram_Y.detach()).mean()


In [ ]:
def tv_loss(Y_hat):
    return 0.5 * (
        # 这里的Tensor是BHWC格式的，通道在最后，所以按照减法应该是第3轴（索引2）的位置进行切片
        torch.abs(Y_hat[:, :, 1:, :] - Y_hat[:, :, :-1, :]).mean()
        + torch.abs(Y_hat[:, :, :, 1:] - Y_hat[:, :, :, :-1]).mean()
    )



In [ ]:
#初始化三个损失的权重
content_weight, style_weight, tv_weight = 1, 10000, 10

#定义计算总权重损失的函数，输入参数：图像，
def compute_loss(X, contents_Y_hat, styles_Y_hat, contents_Y, styles_Y_gram):
    # 分别计算内容损失、样式损失和总变差损失
    contents_l = [
        content_loss(Y_hat, Y) * content_weight
        for Y_hat, Y in zip(contents_Y_hat, contents_Y)
    ]
    styles_l = [
        style_loss(Y_hat, Y) * style_weight
        for Y_hat, Y in zip(styles_Y_hat, styles_Y_gram)
    ]
    tv_l = tv_loss(X) * tv_weight
    # 对所有损失求和
    l = sum(10 * styles_l + contents_l + [tv_l])
    return contents_l, styles_l, tv_l, l


In [ ]:
#定义一个继承了父类nn.Module的子类SynthesizedImage
class SynthesizedImage(nn.Module):
    #初始化类的函数，self是自身传入参数不用管，img_shape图像的大小，**表示字典变量
    def __init__(self, img_shape, **kwargs):
        super(SynthesizedImage, self).__init__(**kwargs)
        #该类的weight的定义，含义是将一个固定不可训练的tensor转换成可以训练的类型parameter，
        # 并将这个parameter绑定到这个module里面(net.parameter()中就有这个绑定的parameter，
        # 所以在参数优化的时候可以进行优化的)，所以经过类型转换这个self.v变成了模型的一部分，
        # 成为了模型中根据训练可以改动的参数了。
        # 使用这个函数的目的也是想让某些变量在学习的过程中不断的修改其值以达到最优化。
        #这里我们要优化的参数就是整个合成图像，那么大小也就是原始图像的大小，或者说输入图像的大小
        self.weight = nn.Parameter(torch.rand(*img_shape))
    
    #前向传播只需要返回权重即可，这个权重是指合成图像的像素的权重
    def forward(self):
        return self.weight


In [ ]:
#定义初始化函数，输入参数：内容图像X，设备CPU or GPU，学习率，风格图像Y
def get_inits(X, device, lr, styles_Y):
    #生成初始的合成图像实例，调用类SynthesizedImage产生
    gen_img = SynthesizedImage(X.shape).to(device)
    #使用copy_（）:
    #解释说明：比如x4.copy_(x2),将x2的数据复制到x4,并且会
    #修改计算图，使得反向传播自动计算梯度时，计算出x4的梯度后
    #再继续前向计算x2的梯度。注意，复制完成之后，两者的值的改变互不影响，
    #因为他们并不共享内存。
    #实例gen_img的权重属性weight的数据data用内容图像X的值复制得到
    gen_img.weight.data.copy_(X.data)
    #训练优化使用Adam优化器，自适应动量优化器，输入参数为合成图像实例的参数方法，学习率由lr输入
    trainer = torch.optim.Adam(gen_img.parameters(), lr=lr)
    #多个风格特征提取得到的styles_Y分别进行Gram求解组成列表赋值给styles_Y_gram
    styles_Y_gram = [gram(Y) for Y in styles_Y]
    #返回生成的图像，风格特征的Gram列表，训练优化器
    return gen_img(), styles_Y_gram, trainer


In [ ]:
#定义训练函数，参数输入：原始合成图像X（这里以原始内容图像content_X为基础），内容特征提取图像，风格特征提取图像，
# 使用设备，初始学习率，迭代次数，学习率开始衰减的迭代次数
def train(X, contents_Y, styles_Y, device, lr, num_epochs, lr_decay_epoch):
    #使用函数get_inits初始化得到：返回生成的图像，风格特征的Gram列表，训练优化器
    X, styles_Y_gram, trainer = get_inits(X, device, lr, styles_Y)
    #使用官方的torch.optim.lr_scheduler.StepLR方法确定训练的步骤，优化器选择，学习率开始衰减的迭代次数，衰减的gamma值
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_decay_epoch, 0.8)
    #画图
    animator = d2l.Animator(
        xlabel="epoch",
        ylabel="loss",
        xlim=[10, num_epochs],
        legend=["content", "style", "TV"],
        ncols=2,
        figsize=(7*1.5, 2.5*1.5),
    )

    #开始迭代循环
    for epoch in range(num_epochs):
        #训练优化器梯度清零
        trainer.zero_grad()
        #提取特征，X是合成图像，会被迭代，然后重新输入训练，即每次迭代都将图像X输入到特征提取中，
        # 计算特征，计算loss，优化参数，再得到新的x，再输入
        contents_Y_hat, styles_Y_hat = extract_features(X, content_layers, style_layers)
        contents_l, styles_l, tv_l, l = compute_loss(
            X, contents_Y_hat, styles_Y_hat, contents_Y, styles_Y_gram
        )
        #将总权重loss反向传播，计算参数的梯度
        l.backward()
        trainer.step()
        scheduler.step()
        #每10次输出一次后处理后的合成图像结果
        if (epoch + 1) % 5 == 0:
            animator.axes[1].imshow(postprocess(X))
            animator.add(
                epoch + 1, [float(sum(contents_l)), float(sum(styles_l)), float(tv_l)]
            )
    return X


In [ ]:
device, image_shape = d2l.try_gpu(), (600, 600)
net = net.to(device)
#获取原始内容图像和获取了内容特征的图像，由于以内容图像为基础的合成图像，所以需要保留content_X
content_X, contents_Y = get_contents(image_shape, device)
#获取风格特征提取后的图像，省略了原始的风格图像style_X
_, styles_Y = get_styles(image_shape, device)
#进行训练，输入原始内容图像作为初次合成图像，内容特征提取图像，风格特征提取图像，
# 使用设备，初始学习率0.3，迭代次数500，学习率开始衰减的迭代次数50
output = train(content_X, contents_Y, styles_Y, device, 0.3, 500, 50)
